In [23]:
import torch
import cv2
from IPython.display import clear_output, Image # to display images

In [26]:
!pip install ultralytics

In [27]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [28]:
if torch.cuda.is_available():
    print(f"CUDA is available! GPU: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA is not available, using CPU.")

CUDA is available! GPU: Tesla T4


In [29]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [30]:
print(torch.version.cuda)  # Shows CUDA version used by PyTorch

12.4


In [31]:
!nvidia-smi # check the installed nvidia drivers

Thu Jan 30 18:00:47 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [32]:
"""
To ensure you are using the cuda chips use:

# Move model and data to GPU
model = model.to(device)
data = data.to(device)

"""

'\nTo ensure you are using the cuda chips use:\n\n# Move model and data to GPU\nmodel = model.to(device)\ndata = data.to(device)\n\n'

In [33]:
import zipfile
import os

zip_path = "/content/gdrive/MyDrive/LicensePlateDataset.zip"
extract_path = "/content/gdrive/MyDrive/LicensePlateDataset_extracted"

# Klasörün var olup olmadığını kontrol et, yoksa oluştur
os.makedirs(extract_path, exist_ok=True)

# ZIP dosyasını aç ve çıkar
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Dosyalar başarıyla çıkarıldı")

Dosyalar başarıyla çıkarıldı


In [34]:
# !cp "/content/drive/MyDrive/traffic_video.mp4" "/content/video.mp4" # Copy the video to the colab workspace

# Alternative: Use the video directly from the drive - Suggested
cap = cv2.VideoCapture("/content/drive/MyDrive/traffic_video.mp4")

In [35]:
from ultralytics import YOLO

In [36]:
# Loading a pretrained model to save up from time
model = YOLO("yolo11n.pt")

# clear_output()
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 2.5.1+cu124 (Tesla T4)


In [38]:
os.listdir(extract_path)

['LicensePlateDataset']

In [ ]:
# Training the model
train_results = model.train(
    data="/content/gdrive/MyDrive/LicensePlateDataset_extracted/LicensePlateDataset/data.yaml", #path to dataset YAML
    epochs=10,
    imgsz=640, # training image size
    device = 0, # device to run on, i.e. device=0 or device=0,1,2,3 or device = cpu
)

Ultralytics 8.3.70 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/content/gdrive/MyDrive/LicensePlateDataset_extracted/LicensePlateDataset/data.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_c

100%|██████████| 755k/755k [00:00<00:00, 20.0MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /content/gdrive/MyDrive/LicensePlateDataset_extracted/LicensePlateDataset/train/labels... 21173 images, 28 backgrounds, 0 corrupt: 100%|██████████| 21173/21173 [08:13<00:00, 42.87it/s] 


train: New cache created: /content/gdrive/MyDrive/LicensePlateDataset_extracted/LicensePlateDataset/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       4/10      2.41G      1.188     0.6401      1.209          5        640: 100%|██████████| 1324/1324 [09:22<00:00,  2.35it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 64/64 [00:31<00:00,  2.06it/s]

                   all       2046       2132      0.954      0.912      0.953      0.635



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       5/10      2.35G      1.174     0.6204      1.211         17        640:   5%|▍         | 61/1324 [00:24<05:50,  3.60it/s]

In [ ]:
test_model = YOLO("runs/detect/train/weights/best.pt")

results = test_model("test_images", conf=0.5, save=True)
# results[0].show()

In [ ]:
# Working with video
results = test_model("test_videos/1.mp4", save=True)